In [4]:
from azureml.core import Workspace,Dataset,Datastore,Experiment,Run,Environment
ws=Workspace.from_config('C:/Users/satya/Downloads/config (9).json')
ws=Workspace.get(name='satya',subscription_id='b0e1c0a0-1b1e-4b1e-8c1a-1c1e1c1e1c1e',resource_group='satya')

In [5]:
Datastore_Register=Datastore.register_azure_blob_container(workspace=ws, datastore_name='bankatastore',account_key='qSK3vCvWiN+s3nNikLNuBKq+0DqIPEv7jJzTsDBtrPLzB1gydm0BPR/hZasc3bvse1Gsmf5kR+Vu+AStjevnPQ==',
container_name='bank',account_name='storagerrg011',create_if_not_exists=True)

In [6]:
dataset=Dataset.Tabular.from_delimited_files(path=[(Datastore_Register,'bank-additional.csv')])

In [7]:
dataset=dataset.register(workspace=ws,name='bankdata',create_new_version=True)

In [8]:
#sanity check
ds1=dataset.to_pandas_dataframe()
#count yes and nos
ds1['y'].value_counts()

no     3668
yes     451
Name: y, dtype: int64

In [9]:
#setup pipelines for autml

from azureml.pipeline.steps import AutoMLStep
from azureml.pipeline.core import Pipeline,PipelineData,TrainingOutput
#import automl config
from azureml.train.automl import AutoMLConfig

In [10]:
#import compute cluster
from azureml.core.compute import ComputeTarget,AmlCompute
cluster_name='cluster1'
cluster_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='lowpriority',max_nodes=4)
cluster=ComputeTarget.create(ws,cluster_name,cluster_config)
cluster.wait_for_completion(show_output=True)


InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
#get compute target
compute_target=ws.compute_targets['cluster1']

In [11]:
Datastore=Datastore.get(ws,'bankatastore')

In [12]:
automl_config=AutoMLConfig(compute_target=cluster,task='classification',primary_metric='AUC_weighted',
experiment_timeout_minutes=30,label_column_name='y',featurization='auto',max_concurrent_iterations=4,training_data=dataset,validation_size=0.25)

#pipeline steps
#storing metrics
metrics_store=PipelineData(name='metrics',datastore=Datastore,pipeline_output_name='metrics_output',training_output=TrainingOutput(type='Metrics'))
model_store=PipelineData(name='model',datastore=Datastore,pipeline_output_name='model',training_output=TrainingOutput(type='Model'))
#automl steps
automl_step=AutoMLStep(name='automl_mod',automl_config=automl_config,outputs=[metrics_store,model_store])
#pipeline final
automl_pipeline=Pipeline(workspace=ws,steps=[automl_step])
experiment_automl_pipeline=Experiment(ws,'automl_pipeline_bank')
experiment_automl_pipeline_run=experiment_automl_pipeline.submit(automl_pipeline)
experiment_automl_pipeline_run.wait_for_completion(show_output=True)

Created step automl_mod [b8435922][30f6715a-62da-427b-be9d-34fcd91ec2fb], (This step will run and generate new outputs)
Submitted PipelineRun e42a0d79-57ea-4aab-a0a8-6926d6803a37
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e42a0d79-57ea-4aab-a0a8-6926d6803a37?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/azuremlsdk-rg01/workspaces/azuremlsdk-ws01&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRunId: e42a0d79-57ea-4aab-a0a8-6926d6803a37
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e42a0d79-57ea-4aab-a0a8-6926d6803a37?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/azuremlsdk-rg01/workspaces/azuremlsdk-ws01&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRun Status: Running


StepRunId: a2a74c4c-53df-4578-aa56-f85d827c2342
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/a2a74c4c-53df-4578-aa56-f85d827c2342?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/a

'Finished'

In [13]:
best_metric=experiment_automl_pipeline_run.get_metrics()

{}

In [14]:
best_metric_model=experiment_automl_pipeline_run.get_pipeline_output('metrics_output')
num_file_downloaded=best_metric_model.download('.',show_progress=True)

Downloaded azureml/a2a74c4c-53df-4578-aa56-f85d827c2342/metrics, 1 files out of an estimated total of 1


In [15]:
best_metric_model

Name,Datastore,Path on Datastore,Produced By PipelineRun
metrics_output,bankatastore,azureml/a2a74c4c-53df-4578-aa56-f85d827c2342/metrics,e42a0d79-57ea-4aab-a0a8-6926d6803a37


In [16]:
import pandas as pd
import json
with open(best_metric_model.path_on_datastore) as f:
    metrics=f.read()
metrics=json.loads(metrics)
metrics=pd.DataFrame(metrics)
metrics

,a2a74c4c-53df-4578-aa56-f85d827c2342_11,a2a74c4c-53df-4578-aa56-f85d827c2342_0,a2a74c4c-53df-4578-aa56-f85d827c2342_41,a2a74c4c-53df-4578-aa56-f85d827c2342_3,a2a74c4c-53df-4578-aa56-f85d827c2342_13,a2a74c4c-53df-4578-aa56-f85d827c2342_6,a2a74c4c-53df-4578-aa56-f85d827c2342_5,a2a74c4c-53df-4578-aa56-f85d827c2342_12,a2a74c4c-53df-4578-aa56-f85d827c2342_15,a2a74c4c-53df-4578-aa56-f85d827c2342_10,...,a2a74c4c-53df-4578-aa56-f85d827c2342_48,a2a74c4c-53df-4578-aa56-f85d827c2342_52,a2a74c4c-53df-4578-aa56-f85d827c2342_57,a2a74c4c-53df-4578-aa56-f85d827c2342_53,a2a74c4c-53df-4578-aa56-f85d827c2342_22,a2a74c4c-53df-4578-aa56-f85d827c2342_44,a2a74c4c-53df-4578-aa56-f85d827c2342_49,a2a74c4c-53df-4578-aa56-f85d827c2342_19,a2a74c4c-53df-4578-aa56-f85d827c2342_58,a2a74c4c-53df-4578-aa56-f85d827c2342_50
precision_score_weighted,[0.8835332515374474],[0.9025759828386043],[0.8946759360951253],[0.8746649529612259],[0.8939470009309749],[0.918758921629331],[0.9009213310072473],[0.8984240052094216],[0.8906041716893426],[0.8901087555801952],...,[0.905914809571871],[0.8988437628068764],[0.9061183235497839],[0.8994536569647461],[0.9062347861772757],[0.8930839999775131],[0.9021714575797958],[0.7926185314355736],[0.7926185314355736],[0.8854676553327084]
f1_score_macro,[0.6573610815414095],[0.7422698838248436],[0.7201235927043328],[0.5133161512027491],[0.5302596976374463],[0.7242275856284799],[0.7323919132279423],[0.7300477337615989],[0.5871840814786873],[0.680466938514432],...,[0.75516607505071],[0.722758604547742],[0.7355789322857587],[0.738843813387424],[0.7546693643134516],[0.7215910763557147],[0.7429721489871866],[0.47098099640472524],[0.47098099640472524],[0.6687506432026346]
accuracy,[0.8058252427184466],[0.9116504854368932],[0.9058252427184466],[0.8932038834951457],[0.8961165048543689],[0.8388349514563107],[0.9116504854368932],[0.9087378640776699],[0.9009708737864077],[0.9058252427184466],...,[0.912621359223301],[0.9106796116504854],[0.916504854368932],[0.9067961165048544],[0.9135922330097087],[0.9019417475728155],[0.9106796116504854],[0.8902912621359224],[0.8902912621359224],[0.9029126213592233]
average_precision_score_macro,[0.7069772272091932],[0.7939039493076803],[0.7935690308972014],[0.7482210024857583],[0.7262358457038626],[0.7763792831272348],[0.7807430394103696],[0.7958024212611701],[0.8208584024233442],[0.7895896213006441],...,[0.7994296569235725],[0.7926972344992931],[0.8141236087485162],[0.7857923756297356],[0.7883751718068863],[0.7793088219170399],[0.7989516875421516],[0.7240829511704214],[0.8084802036362877],[0.7672606229224039]
balanced_accuracy,[0.747406413757829],[0.713730807461808],[0.6910616573860511],[0.5210333812644156],[0.5304281950569866],[0.8590536667277868],[0.6982127174993487],[0.7004564711786221],[0.5641906563341408],[0.6406278650080582],...,[0.7336736761853293],[0.686028893757057],[0.6931799538703545],[0.718763571090802],[0.7303394099651614],[0.7005191997761071],[0.7170650736819757],[0.5],[0.5],[0.6312330512154872]
recall_score_micro,[0.8058252427184466],[0.9116504854368932],[0.9058252427184466],[0.8932038834951457],[0.8961165048543689],[0.8388349514563107],[0.9116504854368932],[0.9087378640776699],[0.9009708737864077],[0.9058252427184466],...,[0.912621359223301],[0.9106796116504854],[0.916504854368932],[0.9067961165048544],[0.9135922330097087],[0.9019417475728155],[0.9106796116504854],[0.8902912621359224],[0.8902912621359224],[0.9029126213592233]
f1_score_micro,[0.8058252427184465],[0.9116504854368932],[0.9058252427184466],[0.8932038834951457],[0.8961165048543689],[0.8388349514563107],[0.9116504854368932],[0.9087378640776699],[0.9009708737864077],[0.9058252427184466],...,[0.912621359223301],[0.9106796116504854],[0.916504854368932],[0.9067961165048544],[0.9135922330097087],[0.9019417475728155],[0.9106796116504854],[0.8902912621359224],[0.8902912621359224],[0.9029126213592235]
f1_score_weighted,[0.833415927538958],[0.9053621038201585],[0.8980784515974763],[0.8489532245687784],[0.8538558175520841],[0.8629991550947087],[0

In [17]:
#publishing the pipeline
published_pipeline=experiment_automl_pipeline_run.publish_pipeline(name='automl_pipeline_bank',description='automl pipeline for bank data',version='1.0')
published_pipeline

Name,Id,Status,Endpoint
automl_pipeline_bank,b890cc4e-cd43-4390-a62d-de641f628238,Active,REST Endpoint


In [18]:
#import interactive auth
from azureml.core.authentication import InteractiveLoginAuthentication
auth=InteractiveLoginAuthentication()
header=auth.get_authentication_header()
endpoint=published_pipeline.endpoint


In [21]:
endpoint

'https://eastus2.api.azureml.ms/pipelines/v1.0/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourceGroups/azuremlsdk-rg01/providers/Microsoft.MachineLearningServices/workspaces/azuremlsdk-ws01/PipelineRuns/PipelineSubmit/b890cc4e-cd43-4390-a62d-de641f628238'

In [25]:
#from azureml.widgets import RunDetails
#import RunDetails
#from azureml.widgets import RunDetails
from azureml.pipeline.core.run import PipelineRun
#pipeline_run=PipelineRun(experiment=experiment_automl_pipeline,id='automl_pipeline_bank_')
#RunDetails(pipeline_run).show()

In [19]:
import requests
response=requests.post(endpoint,headers=header,json={'ExperimentName': 'automl_pipeline_bank_exp1'})

In [23]:
runid=response.json()['Id']
runid

'53eedc73-de58-48eb-ac82-f3b42ab6d630'

In [26]:
#deploy the model with aci
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import InferenceConfig,Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
#import aci compute



In [24]:
#register the model
#geting the run id of the best model
run_id=ws.get_run('361264f0-0f0c-453e-8209-f5db502d6d78_49')
model=run_id.register_model(model_name='automl_model',model_path='outputs/model.pkl')

In [ ]:
azureml-train-automl-runtime==1.48.0
  - inference-schema
  - azureml-interpret==1.48.0
  - azureml-defaults==1.48.0
- numpy==1.21.6
- pandas==1.1.5
- scikit-learn==0.22.1
- py-xgboost==1.3.3
- fbprophet==0.7.1
- holidays==0.10.3
- psutil==5.9.3

In [43]:
env=Environment('automl_env')
dep=CondaDependencies.create(conda_packages=['numpy','pandas','scikit-learn','pip','psutil==5.9.3','inference-schema','azureml-interpret==1.48.0','azureml-defaults==1.48.0',' numpy==1.21.6','py-xgboost==1.3.3','fbprophet==0.7.1','holidays==0.10.3'])
env.python.conda_dependencies=dep
env.register(ws)

{
    "assetId": "azureml://locations/eastus2/workspaces/d63d0fb7-836f-4451-a68b-77cc137d7ddb/environments/automl_env/versions/3",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220708.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "automl_env",
    "python":

In [27]:
#compute=ComputeTarget(ws,'cluster1')
aci_config=AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,enable_app_insights=True,auth_enabled=True)
#creating inference config

inference_config=InferenceConfig(entry_script='scoring_file_v_2_0_0.py')
#deploying the model
model=Model(ws,'automl_model')
service=model.deploy(workspace=ws,name='automlservicebankdata',models=[model],inference_config=inference_config,deployment_config=aci_config,overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-01-04 13:23:58-05:00 Creating Container Registry if not exists.
2023-01-04 13:23:58-05:00 Generating deployment configuration.
2023-01-04 13:23:59-05:00 Submitting deployment to compute.
2023-01-04 13:24:03-05:00 Checking the status of deployment automlservicebankdata..
2023-01-04 13:26:17-05:00 Checking the status of inference endpoint automlservicebankdata.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [27]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://ac9423f2-5309-43bd-8c38-496460e445e2.eastus2.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = '0j7oMCH6D3zgN7A4hve3pWlX3GO2tkFt'

# Two sets of data to score, so we get two results back
data={
  "Inputs": {
    "data": [
      {
       "age": 17,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 971,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 0
      },
        {
            "age": 87,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.8930000,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 47100000000,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 509900000000000,
            "pdays": 10,
            "poutcome": "failure",
            "previous": 0
      }
    ]
  }
  }

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

ConnectionError: HTTPConnectionPool(host='ac9423f2-5309-43bd-8c38-496460e445e2.eastus2.azurecontainer.io', port=80): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EE72D57208>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [28]:
service.get_logs()

'2023-01-04T18:26:07,701718700+00:00 - rsyslog/run \n2023-01-04T18:26:07,701727100+00:00 - iot-server/run \n2023-01-04T18:26:07,708868100+00:00 - gunicorn/run \n2023-01-04T18:26:07,710942400+00:00 | gunicorn/run | \n2023-01-04T18:26:07,716578700+00:00 | gunicorn/run | ###############################################\n2023-01-04T18:26:07,720079700+00:00 | gunicorn/run | AzureML Container Runtime Information\n2023-01-04T18:26:07,725159600+00:00 | gunicorn/run | ###############################################\n2023-01-04T18:26:07,729368400+00:00 | gunicorn/run | \n2023-01-04T18:26:07,733801500+00:00 | gunicorn/run | \n2023-01-04T18:26:07,733802000+00:00 - nginx/run \n2023-01-04T18:26:07,748784800+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20221129.v2\n2023-01-04T18:26:07,750299800+00:00 | gunicorn/run | \n2023-01-04T18:26:07,752051500+00:00 | gunicorn/run | \n2023-01-04T18:26:07,757539700+00:00 | gunicorn/run | PATH environment variable